In [8]:
from PIL import Image 
import numpy as np 

filename = 'cat1.jpg'
im = Image.open(filename)
im_gray = im.convert('L')
im_arr = np.asarray(im)
im_arr_gray = np.asarray(im_gray)
im_arr.shape, im_arr_gray.shape

((408, 612, 3), (408, 612))

In [12]:
im_arr_gray

array([[255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       ...,
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255]], dtype=uint8)

In [ ]:
import os 

def load_image(filepath):
    im = Image.load(filepath)
    arr = np.asarray(im)
    return arr

def load_images(directory):
    files = os.listdir(directory)
    images = []
    for filename in files:
        filepath = f"{directory}/{filename}"
        try:
            image = load_image(filepath)
            images.append(image)
        except Exception as e:
            print(f"--Failed to load file: {filename}")
    return images